# Semi-structured RAG

Let's evaluate your architecture on a small semi-structured Q&A dataset. This dataset is composed of QA pairs over pdfs that contain tables.

## Pre-requisites

We will install quite a few prerequisites for this example since we are comparing various techinques and models.

In [ ]:
%pip install -U langchain langsmith langchainhub  langchain_benchmarks langchain_experimental
%pip install --quiet chromadb openai huggingface pandas "unstructured[all-docs]"

For this code to work, please configure LangSmith environment variables with your credentials.

In [1]:
import os

# os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = "sk-..."  # Your API key

# Silence warnings from HuggingFace
os.environ["TOKENIZERS_PARALLELISM"] = "false"

## Review Q&A Tasks

The registry provides configurations to test out common architectures on curated datasets.

In [2]:
from langchain_benchmarks import clone_public_dataset, registry

In [3]:
registry = registry.filter(Type="RetrievalTask")
registry

Name,Type,Dataset ID,Description
LangChain Docs Q&A,RetrievalTask,452ccafc-18e1-4314-885b-edd735f17b9d,Questions and answers based on a snapshot of the LangChain python docs. The environment provides the documents and the retriever information. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Semi-structured Earnings,RetrievalTask,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).


In [4]:
task = registry["Semi-structured Earnings"]
task

Name,Semi-structured Earnings
Type,RetrievalTask
Dataset ID,c47d9617-ab99-4d6e-a6e6-92b8daf85a7d
Description,Questions and answers based on PDFs containing tables and charts. The task provides the raw documents as well as factory methods to easily index them and create a retriever. Each example is composed of a question and reference answer. Success is measured based on the accuracy of the answer relative to the reference answer. We also measure the faithfulness of the model's response relative to the retrieved documents (if any).
Retriever Factories,"basic, parent-doc, hyde"
Architecture Factories,
get_docs,<function load_docs at 0x13fca0180>


In [5]:
clone_public_dataset(task.dataset_id, dataset_name=task.name)

Dataset Semi-structured Earnings already exists. Skipping.
You can access the dataset at https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/f8f24935-cf57-4cb3-a30f-8df303a46962.


### Now, index the documents

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="thenlper/gte-base")

# Arguments to pass to partition_pdf
unstructured_config = {
    # Unstructured first finds embedded image blocks
    "extract_images_in_pdf": False,
    # Use layout model (YOLOX) to get bounding boxes (for tables) and find titles
    # Titles are any sub-section of the document
    "infer_table_structure": True,
    # Post processing to aggregate text once we have the title
    "chunking_strategy": "by_title",
    # Chunking params to aggregate text blocks
    # Attempt to create a new chunk 3800 chars
    # Attempt to keep chunks > 2000 chars
    "max_characters": 4000,
    "new_after_n_chars": 3800,
    "combine_text_under_n_chars": 2000,
}
docs = list(task.get_docs(unstructured_config=unstructured_config))
retriever_factory = task.retriever_factories["basic"]
# Indexes the documents with the specified embeddings
retriever = retriever_factory(embeddings, docs=docs)

Some weights of the model checkpoint at microsoft/table-transformer-structure-recognition were not used when initializing TableTransformerForObjectDetection: ['model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TableTransformerForObjectDetection from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/26 [00:00<?, ?it/s]

### Time to evaluate

We will compose our retriever with a simple Llama based LLM.

In [7]:
from langchain.chat_models import ChatAnthropic
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer based solely on the retrieved documents below:\n\n<Documents>\n{docs}</Documents>",
        ),
        ("user", "{Question}"),
    ]
)
llm = ChatAnthropic(model="claude-2")


def create_chain(retriever):
    return (
        RunnableAssign({"docs": (lambda x: next(iter(x.values()))) | retriever})
        | prompt
        | llm
        | StrOutputParser()
    )

In [8]:
from functools import partial

from langsmith.client import Client

from langchain_benchmarks.rag import get_eval_config

client = Client()
RAG_EVALUATION = get_eval_config()
chain = create_chain(retriever)
test_run = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=chain,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-roasted-house-33' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/47229cb0-714a-41d6-9517-d36652dc2f63?eval=true

View all tests for Dataset Semi-structured Earnings at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/f8f24935-cf57-4cb3-a30f-8df303a46962
[------------------------------------------------->] 5/5
 Eval quantiles:
                               0.25      0.5      0.75      mean      mode
embedding_cosine_distance  0.050985  0.07845  0.086445  0.069356  0.038448
score_string:accuracy      0.100000  1.00000  1.000000  0.640000  1.000000


## Example processing the docs

RAG apps are as good as the information they are able to retrieve. Let's try indexing the tables' summaries to
improve the likelihood that they are retrieved whenever a user asks a relevant question.

We will use unstructured's `partition_pdf` functionality and generate summaries using an LLM.

You can define your own indexing pipeline to see how it impacts the downstream performance.

In [9]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.document import Document
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable.passthrough import RunnableAssign

# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are summarizing semi-structured tables or text in a pdf.\n\n```document\n{doc}\n```",
        ),
        ("user", "Write a concise summary."),
    ]
)

# Summary chain
model = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k")


def create_doc(x) -> Document:
    return Document(
        page_content=x["output"],
        metadata=x["doc"].metadata,
    )


summarize_chain = (
    {"doc": lambda x: x}
    | RunnableAssign({"prompt": prompt})
    | {
        "output": itemgetter("prompt") | model | StrOutputParser(),
        "doc": itemgetter("doc"),
    }
    | create_doc
)

In [10]:
summaries = summarize_chain.batch(
    [doc for doc in docs if doc.metadata["element_type"] == "table"]
)

Index the documents and create the retriever. We will re

In [11]:
# Indexes the documents with the specified embeddings
retriever_with_summaries = retriever_factory(
    embeddings,
    docs=docs + summaries,
    # Specify a unique transformation name to avoid local cache collisions with other indices.
    transformation_name="table-summaries",
)

  0%|          | 0/36 [00:00<?, ?it/s]

### Evaluate

We'll evaluate the new chain on the same dataset.

In [12]:
chain_2 = create_chain(retriever_with_summaries)

test_run_with_summaries = client.run_on_dataset(
    dataset_name=task.name,
    llm_or_chain_factory=chain_2,
    evaluation=RAG_EVALUATION,
    verbose=True,
)

View the evaluation results for project 'test-proper-underwear-26' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/154950c6-420b-406b-9ced-66edba4c9ff6?eval=true

View all tests for Dataset Semi-structured Earnings at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/datasets/f8f24935-cf57-4cb3-a30f-8df303a46962
[>                                                 ] 0/5

Chain failed for example e04fac04-a465-4912-b91b-950b1dba9ac3 with inputs {'Question': 'Analyzing the operating expenses for Q3 2023, which category saw the largest increase when compared to Q3 2022?'}
Error Type: KeyError, Message: 'doc'
Chain failed for example 68183aca-7d8a-43d2-8b96-3ae2be5d7564 with inputs {'Question': 'How many bank failures were there between 2021 and 2023?'}
Error Type: KeyError, Message: 'doc'
Chain failed for example eec2946e-5504-4acd-af88-2ffd37e4b4ac with inputs {'Question': "Based on the net income (loss) reported for Q3 2023, how did Datadog's performance change compared to Q3 2022, and what might this indicate in terms of their financial recovery?"}
Error Type: KeyError, Message: 'doc'
Chain failed for example 05400fc8-1b72-4201-8cd8-4e56f02025fc with inputs {'Question': 'What is the change in cash flow from operations for Q3 2023 versus Q3 2022?'}
Error Type: KeyError, Message: 'doc'


[--------------------------------------->          ] 4/5

Chain failed for example 6a925751-f04c-41fd-b7b6-f1968499910c with inputs {'Question': 'What is the difference in deposits for bank failures in 2001-2020 versus 2021-2023?'}
Error Type: KeyError, Message: 'doc'


[------------------------------------------------->] 5/5